In [ ]:
from utils.measurement_utils import load_json, dump_json
from utils.common import compute_geo_info, compute_rtts_per_dst_src, compute_remove_wrongly_geolocated_probes
from default import ANCHORS_FILE, PROBES_FILE, PROBES_AND_ANCHORS_FILE, REMOVED_PROBES_FILE, ANCHORS_MESHED_PING_TABLE, PROBES_TO_ANCHORS_PING_TABLE, PAIRWISE_DISTANCE_FILE

## Find probes that disrespect the internet speed rule

In [ ]:
anchors = load_json(ANCHORS_FILE)
probes = load_json(PROBES_FILE)
all_probes = load_json(PROBES_AND_ANCHORS_FILE)

In [ ]:
vp_coordinates_per_ip, ip_per_coordinates, country_per_vp, asn_per_vp, \
    vp_distance_matrix, probe_per_ip = compute_geo_info(
        all_probes, serialized_file=PAIRWISE_DISTANCE_FILE)

In [ ]:
removed_probes = set()


rtt_per_srcs_dst = compute_rtts_per_dst_src(
    ANCHORS_MESHED_PING_TABLE, filter, threshold=300)
removed_anchors = compute_remove_wrongly_geolocated_probes(rtt_per_srcs_dst,
                                                            vp_coordinates_per_ip,
                                                            vp_distance_matrix,
                                                            removed_anchors=set())

# Remove the probes to compute the removed probes
vp_coordinates_per_ip = {ip: vp_coordinates_per_ip[ip]
                            for ip in vp_coordinates_per_ip
                            if ip not in removed_anchors}

rtt_per_srcs_dst = compute_rtts_per_dst_src(
    PROBES_TO_ANCHORS_PING_TABLE, filter, threshold=300)
removed_probes = compute_remove_wrongly_geolocated_probes(rtt_per_srcs_dst,
                                                            vp_coordinates_per_ip,
                                                            vp_distance_matrix,
                                                            removed_anchors)

removed_probes.update(removed_anchors)

print(f"Removing {len(removed_probes)} probes")
dump_json(list(removed_probes), REMOVED_PROBES_FILE)